# 处理速度滑冰数据

In [126]:
import pandas as pd
import os
import numpy as np
import datetime as dt

In [137]:
# 选择参数
# 选择直道或者弯道
road_type = input("请选择直道或者弯道：")
print("您选择的是：", road_type)


您选择的是： 弯道


In [128]:
# 填写0，或者1
traget_datas = ["Joint Angles ZXY", "Segment Angular Velocity"]
i = input("请选择0 [Joint Angles ZXY] 或1 [Segment Angular Velocity]")
target_data = traget_datas[int(i)]
print(f"您选择的是：{i}", target_data)
event_name = "蹬冰离冰"

您选择的是：0 Joint Angles ZXY


In [138]:
# 文件路径设置
excel_dir = "/Volumes/ESD-USB/哈尔滨滑冰测试/excel/蹬伸角_蹬伸半径"
record_excel_path = "/Volumes/ESD-USB/哈尔滨滑冰测试/RecordData/20231201_短道速滑特征时刻表_期刊投稿.xlsx"

In [139]:
# 读取记录数据的excel 表格
excelfile = pd.ExcelFile(record_excel_path)
excelfile.sheet_names
record_df = pd.read_excel(record_excel_path, sheet_name="特征时刻")

In [140]:
# # 构建表格表头
# joints = [" Hip ", " Knee ", " Ankle "]
# segments = [" Upper Leg ", " Lower Leg ", " Foot "]
# seg_types = ["Left", "Right"]
# seg_axis = ["x", "y", "z"]
# joint_orientations = ["Flexion/Extension","Abduction/Adduction", "Internal/External Rotation"]
# joint_orientations_ankle = ["Dorsiflexion/Plantarflexion","Abduction/Adduction", "Internal/External Rotation"]

# # 构建列表头名称
# def CreateSelectColumns(seg_type,target_data, segments, joints, joint_orientations, joint_orientations_ankle, seg_axis):
#     if target_data == "Joint Angles ZXY":
#         select_seg_types_colmuns = [seg_type + joint + seg_orientation
#                     for joint in joints
#                     for seg_orientation in ((joint_orientations_ankle if joint.strip()=="Ankle" else joint_orientations))]
#     else:
#         select_seg_types_colmuns = [seg_type + segment + seg_ax
#                     for segment in segments
#                     for seg_ax in seg_axis]
#     return select_seg_types_colmuns



In [141]:
# 获取列表名称
Left_select_columns = ["Left Slip Angle", "Left Slip Radius"]
Right_select_columns =["Right Slip Angle", "Right Slip Radius"]


In [142]:
Left_select_columns

['Left Slip Angle', 'Left Slip Radius']

In [143]:
def GetSlipMinData(slip_times, target_data_df, Left_select_columns, Right_select_columns):
    # 计算滑行过程中的极值
    # 左侧数据极值
    left_max_data_list = []
    for column_name in Left_select_columns:
        if column_name == "Left Slip Angle":
            slip_min = target_data_df.loc[slip_times[0]:slip_times[1], column_name].min()
            left_max_data_list.append(slip_min)
        else:
            slip_max = target_data_df.loc[slip_times[0]:slip_times[1], column_name].max()
            left_max_data_list.append(slip_max)
    
    # 右侧数据极值
    right_max_data_list = []
    for column_name in Right_select_columns:
        if column_name == "Right Slip Angle":
            slip_min = target_data_df.loc[slip_times[2]:slip_times[3], column_name].min()
            left_max_data_list.append(slip_min)
        else:
            slip_max = target_data_df.loc[slip_times[2]:slip_times[3], column_name].max()
            right_max_data_list.append(slip_max)

    total_max_data_list = left_max_data_list + right_max_data_list

    max_data_df = pd.DataFrame([total_max_data_list], columns=Left_select_columns+Right_select_columns)
    return max_data_df

In [144]:


# 读取记录数据的excel表格，获取每行的相关信息

all_persons_df = pd.DataFrame()
for record_row, row in record_df.iterrows():
    date = row["日期"]
    test_number = row["测试序号"]
    name = row["姓名"]
    xsens_number = row["实际xsens号"]

    # 拼接文件名
    excel_fname = f"{int(date)}_{int(test_number)}_{name}-00{int(xsens_number)}_蹬伸角_半径.xlsx"

    excel_path = os.path.join(excel_dir, excel_fname)
    #target_data_df = pd.read_excel(excel_path, sheet_name=target_data)
    target_data_df = pd.read_excel(excel_path)
    # angle_df 
    # 读取record_df一行数据得到的结果
    attribute_info = record_df.loc[record_row, ["姓名", "穿鞋", "xsens号", "实际xsens号"]]
    # 将 series 转换为 dataframe 并重置索引
    attribute_info_df = pd.DataFrame(attribute_info).T.reset_index(drop=True)
    # 获取蹬冰开始和结束时间
    slip_times_1 = [row[f"1{road_type}蹬冰开始-左"], row[f"1{road_type}蹬冰结束-左"], 
                    row[f"1{road_type}蹬冰开始-右"], row[f"1{road_type}蹬冰结束-右"]]
    SlipMaxData_df = GetSlipMinData(slip_times_1, target_data_df, Left_select_columns, Right_select_columns)
    result_df_1 = pd.concat([attribute_info_df, SlipMaxData_df],axis=1 )

    slip_times_2 = [row[f"2{road_type}蹬冰开始-左"], row[f"2{road_type}蹬冰结束-左"], 
                    row[f"2{road_type}蹬冰开始-右"], row[f"2{road_type}蹬冰结束-右"]]
    SlipMaxData_df = GetSlipMinData(slip_times_2, target_data_df, Left_select_columns, Right_select_columns)
    
    result_df_2 = pd.concat([attribute_info_df, SlipMaxData_df],axis=1)

    # 拼接第一和第二道的数据
    total_result_df = pd.concat([result_df_1, result_df_2], axis=0,ignore_index=True)
    # 拼接所有人的数据
    all_persons_df = pd.concat([all_persons_df, total_result_df], axis=0,ignore_index=True)

In [145]:
# 获取当前时间
current_time = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
# 设置路径
result_excel_dir = "/Volumes/ESD-USB/哈尔滨滑冰测试/ResultData"
result_excel_fname = f"{current_time}_{road_type}_{event_name}_蹬伸角_半径raw.xlsx"
result_excel_path = os.path.join(result_excel_dir, result_excel_fname)
# 保存到 Excel（转置后再保存）
all_persons_df.to_excel(result_excel_path, index=False)
print(f"已完成：{result_excel_fname}")

已完成：20231203_222819_弯道_蹬冰离冰_蹬伸角_半径raw.xlsx
